In [1]:
import warnings
warnings.filterwarnings('ignore')

import configparser
cfg = configparser.ConfigParser()
cfg.optionxform = str
cfg.read('/home/sarth/rootdir/datadir/assets/defaults.ini')
cfg = {s: dict(cfg.items(s)) for s in cfg.sections()}
PATHS = cfg['PATHS']

import torch
from torch import nn
import numpy as np
import pandas as pd
import xarray as xr
import networkx as nx
import matplotlib.pyplot as plt
import random
import os
import tqdm
import glob
import itertools
import pickle
import gc
import json
from torch.utils.data import Dataset, DataLoader

RUN_NAME = 'Run03'
PROJECT_DIR = os.path.join('/home/sarth/rootdir/workdir/projects/lumped_analysis_Jun2025')
ANALYSIS_DIR = os.path.join(PROJECT_DIR, 'analysis', RUN_NAME)
RESULTS_DIR = os.path.join(PROJECT_DIR, 'Results', RUN_NAME)

In [2]:
from scipy import stats, signal

def _mask_valid(pred, true):
    mask = ~np.isnan(true) & ~np.isnan(pred)
    pred = pred[mask]
    true = true[mask]
    pred[pred < 0] = 0
    true[true < 0] = 0
    return pred, true

def RMSE(pred, true):
    pred, true = _mask_valid(pred, true)
    return np.sqrt(np.mean((true - pred)**2))

def pearsonr(pred, true):
    pred, true = _mask_valid(pred, true)
    r, _ = stats.pearsonr(true, pred)
    return r

def NSE(pred, true):
    pred, true = _mask_valid(pred, true)
    return 1 - np.sum((true - pred)**2) / np.sum((true - np.mean(true))**2)

def KGE(pred, true):
    pred, true = _mask_valid(pred, true)
    r = pearsonr(pred, true)
    alpha = np.std(pred) / np.std(true)
    beta = np.mean(pred) / np.mean(true)
    return 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)

def PBIAS(pred, true):
    pred, true = _mask_valid(pred, true)
    return np.sum(true - pred) / np.sum(true) * 100

def alpha_NSE(pred, true):
    pred, true = _mask_valid(pred, true)
    return np.std(pred) / np.std(true)

def beta_NSE(pred, true):
    pred, true = _mask_valid(pred, true)
    return (np.mean(pred) - np.mean(true)) / np.std(true)

def _get_fdc(data):
    data = np.sort(data)[::-1]
    return data

def fdc_fms(pred, true, lower = 0.2, upper = 0.7):
    pred, true = _mask_valid(pred, true)

    sim = _get_fdc(pred)
    obs = _get_fdc(true)
    sim[sim <= 0] = 1e-6
    obs[obs <= 0] = 1e-6

    qsm_lower = np.log(sim[np.round(lower * len(sim)).astype(int)])
    qsm_upper = np.log(sim[np.round(upper * len(sim)).astype(int)])
    qom_lower = np.log(obs[np.round(lower * len(obs)).astype(int)])
    qom_upper = np.log(obs[np.round(upper * len(obs)).astype(int)])

    fms = ((qsm_lower - qsm_upper) - (qom_lower - qom_upper)) / (qom_lower - qom_upper + 1e-6)

    return fms * 100

def fdc_fhv(pred, true, h = 0.02):
    pred, true = _mask_valid(pred, true)

    sim = _get_fdc(pred)
    obs = _get_fdc(true)

    obs = obs[:np.round(h * len(obs)).astype(int)]
    sim = sim[:np.round(h * len(sim)).astype(int)]

    fhv = np.sum(sim - obs) / np.sum(obs)

    return fhv * 100

def fdc_flv(pred, true, l = 0.3):
    pred, true = _mask_valid(pred, true)

    sim = _get_fdc(pred)
    obs = _get_fdc(true)
    sim[sim <= 0] = 1e-6
    obs[obs <= 0] = 1e-6

    obs = obs[-np.round(l * len(obs)).astype(int):]
    sim = sim[-np.round(l * len(sim)).astype(int):]

    # transform values to log scale
    obs = np.log(obs)
    sim = np.log(sim)

    # calculate flv part by part
    qsl = np.sum(sim - sim.min())
    qol = np.sum(obs - obs.min())

    flv = -1 * (qsl - qol) / (qol + 1e-6)

    return flv * 100

def mean_peak_timing(pred, true, window = 3):
    pred, true = _mask_valid(pred, true)

    peaks, _ = signal.find_peaks(true, distance=2*window, prominence=np.std(true))

    # pred_idx_lst = []
    timing_error_lst = []
    for idx in peaks:
        if (pred[idx] > pred[idx - 1]) and (pred[idx] > pred[idx + 1]):
            peak_pred = pred[idx]
            peak_pred_idx = idx
        else:
            peak_pred_idx = np.argmax(pred[max(idx - window,0):idx + window + 1]) + max(idx - window,0)
            peak_pred = pred[peak_pred_idx]
        # pred_idx_lst.append(peak_pred_idx)
    
        peak_true = true[idx]
        timing_error = np.abs(peak_pred_idx - idx) 
        timing_error_lst.append(timing_error)
    
    mean_timing_error = np.mean(timing_error_lst) if len(timing_error_lst) > 0 else np.nan

    return mean_timing_error

def missed_peaks(pred, true, window = 3, threshold = 80):
    pred, true = _mask_valid(pred, true)

    peaks_obs_times, _ = signal.find_peaks(true, distance=2*window, height = np.percentile(true, threshold))
    peaks_sim_times, _ = signal.find_peaks(pred, distance=2*window, height = np.percentile(pred, threshold))
    
    missed_events = 0
    for idx in peaks_obs_times:
        nearby_peak_sim_index = np.where(np.abs(peaks_sim_times - idx) <= window)[0]
        if len(nearby_peak_sim_index) == 0:
            missed_events += 1
            # print(idx)
    
    missed_peak_values = (missed_events / len(peaks_obs_times)) * 100 if len(peaks_obs_times) > 0 else np.nan

    return missed_peak_values

def F1_score_of_capturing_peaks(pred, true, window = 3, threshold = 80):
    pred, true = _mask_valid(pred, true)

    peaks_obs_times, _ = signal.find_peaks(true, distance=2*window, height = np.percentile(true, threshold))
    peaks_sim_times, _ = signal.find_peaks(pred, distance=2*window, height = np.percentile(pred, threshold))
    
    true_positive_peaks = 0 # peak in obs and nearby in sim
    true_negative_peaks = 0 # no peak in obs and sim
    false_positive_peaks = 0 # peak in sim but not nearby in obs
    false_negative_peaks = 0 # peak in obs but not nearby in sim

    for idx in peaks_obs_times:
        nearby_peak_sim_index = np.where(np.abs(peaks_sim_times - idx) <= window)[0]
        if len(nearby_peak_sim_index) > 0:
            true_positive_peaks += 1
        else:
            false_negative_peaks += 1
    
    for idx in peaks_sim_times:
        nearby_peak_obs_index = np.where(np.abs(peaks_obs_times - idx) <= window)[0]
        if len(nearby_peak_obs_index) == 0:
            false_positive_peaks += 1

    precision = true_positive_peaks / (true_positive_peaks + false_positive_peaks) if (true_positive_peaks + false_positive_peaks) > 0 else np.nan
    recall = true_positive_peaks / (true_positive_peaks + false_negative_peaks) if (true_positive_peaks + false_negative_peaks) > 0 else np.nan
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else np.nan

    return f1_score



In [3]:
import numpy as np
import xarray as xr

def compute_metrics_ds(y_pred, y_true):
    """
    Compute a set of metrics for each lead time and catchment.
    y_pred and y_true: numpy arrays of shape (time_idx, lead_time, catmt_idx)
    
    Returns:
      xr.Dataset with dims ("time_idx", "lead_time", "catmt_idx") and variables:
          - RMSE, PearsonR, NSE, KGE, PBIAS, alpha_NSE, beta_NSE,
            FDC_FMS, FDC_FHV, FDC_FLV, mean_peak_timing, missed_peaks, F1_score
          - Also includes raw y_pred and y_true.
    """

    time_steps, num_leadtimes, num_catmt = y_pred.shape

    # Prepare arrays to hold computed metrics [lead_time, catmt_idx]
    rmse_arr            = np.empty((num_leadtimes, num_catmt))
    pearson_arr         = np.empty((num_leadtimes, num_catmt))
    nse_arr             = np.empty((num_leadtimes, num_catmt))
    kge_arr             = np.empty((num_leadtimes, num_catmt))
    pbias_arr           = np.empty((num_leadtimes, num_catmt))
    alpha_nse_arr       = np.empty((num_leadtimes, num_catmt))
    beta_nse_arr        = np.empty((num_leadtimes, num_catmt))
    fdc_fms_arr         = np.empty((num_leadtimes, num_catmt))
    fdc_fhv_arr         = np.empty((num_leadtimes, num_catmt))
    fdc_flv_arr         = np.empty((num_leadtimes, num_catmt))
    mean_peak_timing_arr= np.empty((num_leadtimes, num_catmt))
    missed_peaks_arr    = np.empty((num_leadtimes, num_catmt))
    f1_score_arr        = np.empty((num_leadtimes, num_catmt))

    # Loop over lead times and catchments, computing metrics from the time series
    for lt in range(num_leadtimes):
        for cat in range(num_catmt):
            pred = y_pred[:, lt, cat]
            true = y_true[:, lt, cat]
            rmse_arr[lt, cat]             = RMSE(pred, true)
            pearson_arr[lt, cat]          = pearsonr(pred, true)
            nse_arr[lt, cat]              = NSE(pred, true)
            kge_arr[lt, cat]              = KGE(pred, true)
            pbias_arr[lt, cat]            = PBIAS(pred, true)
            alpha_nse_arr[lt, cat]        = alpha_NSE(pred, true)
            beta_nse_arr[lt, cat]         = beta_NSE(pred, true)
            fdc_fms_arr[lt, cat]          = fdc_fms(pred, true)
            fdc_fhv_arr[lt, cat]          = fdc_fhv(pred, true)
            fdc_flv_arr[lt, cat]          = fdc_flv(pred, true)
            mean_peak_timing_arr[lt, cat] = mean_peak_timing(pred, true)
            missed_peaks_arr[lt, cat]     = missed_peaks(pred, true)
            f1_score_arr[lt, cat]         = F1_score_of_capturing_peaks(pred, true)

    # Create coordinates
    lead_times = np.arange(num_leadtimes)
    catmt_idx  = np.arange(num_catmt)
    time_idx   = np.arange(time_steps)

    ds = xr.Dataset(
        {
            "RMSE":             (("lead_time", "catmt_idx"), rmse_arr), # SI
            "PearsonR":         (("lead_time", "catmt_idx"), pearson_arr), # SI
            "NSE":              (("lead_time", "catmt_idx"), nse_arr), # Main
            "KGE":              (("lead_time", "catmt_idx"), kge_arr), # SI
            "PBIAS":            (("lead_time", "catmt_idx"), pbias_arr), # SI
            "alpha_NSE":        (("lead_time", "catmt_idx"), alpha_nse_arr),
            "beta_NSE":         (("lead_time", "catmt_idx"), beta_nse_arr),
            "FDC_FMS":          (("lead_time", "catmt_idx"), fdc_fms_arr),
            "FDC_FHV":          (("lead_time", "catmt_idx"), fdc_fhv_arr), # SI
            "FDC_FLV":          (("lead_time", "catmt_idx"), fdc_flv_arr), # SI
            "mean_peak_timing": (("lead_time", "catmt_idx"), mean_peak_timing_arr), # SI
            "missed_peaks":     (("lead_time", "catmt_idx"), missed_peaks_arr), # SI
            "F1_score":         (("lead_time", "catmt_idx"), f1_score_arr), # Main
            "y_pred":           (("time_idx", "lead_time", "catmt_idx"), y_pred),
            "y_true":           (("time_idx", "lead_time", "catmt_idx"), y_true)
        },
        coords={
            "time_idx": time_idx,
            "lead_time": lead_times,
            "catmt_idx": catmt_idx
        }
    )
    
    return ds

# # Example usage:
# metrics_ds = compute_metrics_ds(y_pred, y_true)
# metrics_ds

In [4]:
HUCs = [str(i).zfill(2) for i in range(1, 18+1)]

REGIONMASKS = {huc: torch.load(f"{PROJECT_DIR}/lumped_training_samples/regionmask/CAMELS-US/HUC{huc}.pt").numpy() for huc in HUCs}

for huc, mask in REGIONMASKS.items():
    print(f"HUC{huc}: {mask.sum()} ({mask.shape[0]})")

HUC01: 12 (395)
HUC02: 39 (395)
HUC03: 45 (395)
HUC04: 16 (395)
HUC05: 26 (395)
HUC06: 10 (395)
HUC07: 26 (395)
HUC08: 10 (395)
HUC09: 5 (395)
HUC10: 52 (395)
HUC11: 26 (395)
HUC12: 28 (395)
HUC13: 4 (395)
HUC14: 5 (395)
HUC15: 12 (395)
HUC16: 6 (395)
HUC17: 52 (395)
HUC18: 21 (395)


In [5]:
from matplotlib import category
import numpy as np
import os
import xarray as xr

import matplotlib.pyplot as plt

In [ ]:
RUN_NAME = 'Run02'
PROJECT_DIR = os.path.join('/home/sarth/rootdir/workdir/projects/lumped_analysis_Jun2025')
ANALYSIS_DIR = os.path.join(PROJECT_DIR, 'analysis', RUN_NAME)
RESULTS_DIR = os.path.join(PROJECT_DIR, 'Results', RUN_NAME)

metrics_dict = {}
for model_huc in tqdm.tqdm(HUCs):
    metrics_dict[model_huc] = {}
    for prediction_huc in HUCs:
        y_true = np.load(os.path.join(ANALYSIS_DIR, 'evaluations', f'HUC{model_huc}', f'HUC{prediction_huc}', 'y_ground_truth.npy'))
        y_pred = np.load(os.path.join(ANALYSIS_DIR, 'evaluations', f'HUC{model_huc}', f'HUC{prediction_huc}', 'y_predictions.npy'))
        y_true = y_true[:, :1, :]
        y_pred = y_pred[:, :1, :]
        metrics = compute_metrics_ds(y_pred, y_true)
        metrics_dict[model_huc][prediction_huc] = metrics

metrics_dict_Run02 = metrics_dict.copy()
del metrics_dict
print(metrics_dict_Run02.keys())

100%|██████████| 18/18 [01:13<00:00,  4.06s/it]


In [10]:
RUN_NAME = 'Run03'
PROJECT_DIR_run03 = os.path.join('/home/sarth/rootdir/workdir/projects/lumped_analysis_Jun2025')
ANALYSIS_DIR_run03 = os.path.join(PROJECT_DIR_run03, 'analysis', RUN_NAME)
RESULTS_DIR_run03 = os.path.join(PROJECT_DIR_run03, 'Results', RUN_NAME)

param_V = 3
param_L = 1
param_C = 1

metrics_dict = {}
y_true = np.load(os.path.join(ANALYSIS_DIR_run03, 'evaluations', f"V{param_V}_L{param_L}_C{param_C}", 'y_ground_truth.npy'))
y_pred = np.load(os.path.join(ANALYSIS_DIR_run03, 'evaluations', f"V{param_V}_L{param_L}_C{param_C}", 'y_predictions.npy'))
y_true = y_true[:, :1, :]
y_pred = y_pred[:, :1, :]
metrics = compute_metrics_ds(y_pred, y_true)
metrics_dict['all'] = metrics

metrics_dict_Run03 = metrics_dict.copy()
del metrics_dict
print(metrics_dict_Run03.keys())

dict_keys(['all'])


In [13]:
metrics_dict_Run03['all']

<xarray.Dataset> Size: 12MB
Dimensions:           (lead_time: 1, catmt_idx: 395, time_idx: 3650)
Coordinates:
  * time_idx          (time_idx) int64 29kB 0 1 2 3 4 ... 3646 3647 3648 3649
  * lead_time         (lead_time) int64 8B 0
  * catmt_idx         (catmt_idx) int64 3kB 0 1 2 3 4 5 ... 390 391 392 393 394
Data variables: (12/15)
    RMSE              (lead_time, catmt_idx) float64 3kB 0.3214 ... 0.2721
    PearsonR          (lead_time, catmt_idx) float64 3kB 0.9873 0.9817 ... 0.741
    NSE               (lead_time, catmt_idx) float64 3kB 0.974 0.9612 ... 0.4904
    KGE               (lead_time, catmt_idx) float64 3kB 0.9558 ... 0.4707
    PBIAS             (lead_time, catmt_idx) float64 3kB -1.009 ... -45.89
    alpha_NSE         (lead_time, catmt_idx) float64 3kB 0.9589 1.013 ... 0.95
    ...                ...
    FDC_FLV           (lead_time, catmt_idx) float64 3kB 6.912 ... -9.336e+11
    mean_peak_timing  (lead_time, catmt_idx) float64 3kB 0.1731 0.64 ... 1.318
    missed_peaks      (lead_time, catmt_idx) float64 3kB 11.5 12.79 ... 34.02
    F1_score          (lead_time, catmt_idx) float64 3kB 0.905 0.8671 ... 0.6337
    y_pred            (time_idx, lead_time, catmt_idx) float32 6MB 0.2706 ......
    y_true            (time_idx, lead_time, catmt_idx) float32 6MB 0.4434 ......

In [12]:
def plot_cdf_pdf_v3(varname='NSE', lead_time=1, suffix='pred_glofas', clip_min_max=[-1, 1]):
    # Colors and transparency for PDF plots
    pdf_color_run02_each = 'blue'
    pdf_color_run02_median = 'darkblue'
    pdf_color_run03 = 'green'
    pdf_alpha = 0.25  # common alpha for all PDF bars

    show_legend = False

    fontsize_label = 16
    fontsize_tick = 12

    # Run02: use variables from global PROJECT_DIR, PATHS, etc.

    grid = np.linspace(clip_min_max[0], clip_min_max[1], 500)
    all_cdfs = []

    for model_huc in HUCs:
        all_metric = []
        for prediction_huc in HUCs:
            ds = metrics_dict_Run02[model_huc][prediction_huc]
            metric_vals = ds.sel(lead_time=lead_time)[varname].values.flatten()
            all_metric.extend(metric_vals[~np.isnan(metric_vals)])
        all_metric = np.array(all_metric)
        cdf_values = np.array([np.mean(all_metric <= x) for x in grid])
        all_cdfs.append(cdf_values)

    all_cdfs = np.array(all_cdfs)  # shape: (n_HUCs, len(grid))
    median_cdf = np.median(all_cdfs, axis=0)

    # Run03
    all_metric = []
    ds = metrics_dict_Run03['all']
    metric_vals = ds.sel(lead_time=lead_time)[varname].values.flatten()
    all_metric.extend(metric_vals[~np.isnan(metric_vals)])
    all_metric = np.array(all_metric)
    cdf_values_run03 = np.array([np.mean(all_metric <= x) for x in grid])
    all_cdfs_run03 = np.array([cdf_values_run03])  # shape: (1, len(grid))

    # Prepare figure and axes
    # fig, ax = plt.subplots(figsize=(8, 6))
    fig, ax = plt.subplots(figsize=(16, 12), dpi=600)
    
    # Plot each HUC's Run02 CDF using lightblue
    for idx, cdf in enumerate(all_cdfs):
        if idx == 0:
            ax.plot(grid, cdf, color='lightblue', linewidth=1, alpha=0.7, label='Regionally each HUC')
        else:
            ax.plot(grid, cdf, color='lightblue', linewidth=1, alpha=0.7)
    
    # Plot median of Run02
    ax.plot(grid, median_cdf, label='Regionally (median)', color='darkblue', linewidth=2)
    # Plot Run03
    ax.plot(grid, all_cdfs_run03[0], label='CAMELS-US', color='green', linestyle='-', linewidth=2, alpha=0.75)

    ax.set_xlabel(varname, fontsize=fontsize_label, fontweight='bold')
    ax.set_ylabel('CDF', fontsize=fontsize_label, fontweight='bold')
    ax.axhline(0.5, color='black', linestyle='--', linewidth=1)
    
    if varname == 'NSE':
        buffer = 0.075
    elif varname == 'PBIAS':
        buffer = 7.5
    else:
        buffer = 0.05
    ax.text(clip_min_max[0] + buffer, 0.51, "CDF=0.5", color='k', fontsize=10, ha='left')
    # Annotate median of Run02 at cdf = 0.5
    med_cdf_at_0_5 = grid[np.abs(median_cdf - 0.5).argmin()]
    ax.scatter(med_cdf_at_0_5, 0.5, color='darkblue')
    ax.text(med_cdf_at_0_5 + buffer, 0.45, f"{med_cdf_at_0_5:.2f}", color='darkblue', fontsize=10, ha='center', fontweight='bold')
    
    # Annotate Run03 at cdf = 0.5
    run03_cdf_at_0_5 = grid[np.abs(all_cdfs_run03[0] - 0.5).argmin()]
    ax.scatter(run03_cdf_at_0_5, 0.5, color='green')
    ax.text(run03_cdf_at_0_5 - buffer, 0.52, f"{run03_cdf_at_0_5:.2f}", color='darkgreen', fontsize=10, ha='center', fontweight='bold')
    
    ax.set_xlim(clip_min_max[0], clip_min_max[1])
    ax.set_ylim(0, 1)
    ax.grid(True, linestyle='--', alpha=0.5)
    if show_legend:
        leg = ax.legend(loc='upper left', title='CDF when trained on', fontsize=fontsize_tick, title_fontsize=fontsize_label)
    ax.tick_params(axis='both', which='major', labelsize=fontsize_tick)
    
    # Create a secondary y-axis for PDF
    ax2 = ax.twinx()
    ax2.set_ylabel('PDF', fontsize=fontsize_label, fontweight='bold')

    # compute width for bar plots based on grid spacing
    bar_width = grid[1] - grid[0]
    
    # Plot PDF for run02 each HUC (using light color)
    final_pdf_values = np.mean([np.gradient(cdf, grid) for cdf in all_cdfs], axis=0)
    # Normalize the pdf before plotting
    # final_pdf_values /= np.trapz(final_pdf_values, grid)
    ax2.bar(grid, final_pdf_values, width=bar_width, color=pdf_color_run02_each, alpha=pdf_alpha, label='Regionally PDF', align='center', zorder=0)
    
    # # Plot PDF for median Run02
    # pdf_median = np.gradient(median_cdf, grid)
    # ax2.bar(grid, pdf_median, width=bar_width, color=pdf_color_run02_median, alpha=pdf_alpha, label='Regionally (median) PDF', align='center')
    
    # Plot PDF for Run03
    pdf_run03 = np.gradient(all_cdfs_run03[0], grid)
    # Normalize the pdf before plotting
    # pdf_run03 /= np.trapz(pdf_run03, grid)
    ax2.bar(grid, pdf_run03, width=bar_width, color=pdf_color_run03, alpha=pdf_alpha, label='CAMELS-US PDF', align='center', zorder=1)
    
    ax2.tick_params(axis='y', which='major', labelsize=fontsize_tick)

    if show_legend:
        # Custom legend handles for PDF: Regionally each HUC, CAMELS-US
        from matplotlib.patches import Patch
        pdf_legend_handles = [
            Patch(facecolor=pdf_color_run02_each, edgecolor='k', alpha=pdf_alpha, label='Regionally each HUC'),
            Patch(facecolor=pdf_color_run03, edgecolor='k', alpha=pdf_alpha, label='CAMELS-US')
        ]
        # Add PDF legend to existing legend
        leg2 = ax.legend(handles=pdf_legend_handles, loc='upper left', fontsize=fontsize_tick, bbox_to_anchor=(0, 0.75))
        # leg2.get_title().set_fontweight('bold')
        ax.add_artist(leg)  # Add the first legend back
    
    plt.show()

    return None


In [ ]:
# plot_cdf_pdf_v3(varname='KGE', lead_time=0, suffix='pred_glofas', clip_min_max=[-1, 1])
# plot_cdf_pdf_v3(varname='PBIAS', lead_time=0, suffix='pred_glofas', clip_min_max=[-100, 100])
# plot_cdf_pdf_v3(varname='PearsonR', lead_time=0, suffix='pred_glofas', clip_min_max=[0, 1])
# plot_cdf_pdf_v3(varname='RMSE', lead_time=0, suffix='pred_glofas', clip_min_max=[0, 3])
# plot_cdf_pdf_v3(varname='FDC_FHV', lead_time=0, suffix='pred_glofas', clip_min_max=[-100, 100])
# plot_cdf_pdf_v3(varname='FDC_FLV', lead_time=0, suffix='pred_glofas', clip_min_max=[-500, 100])